Saving toluene.npz to toluene.npz


In [2]:
import numpy as np

data = np.load('/Users/carolina/Downloads/toluene.npz')
   
E = data['E'] 
R = data['R']

E -= E.min()

print(E.shape, R.shape)

(22140, 1) (22140, 15, 3)


In [3]:
for k in data.iterkeys():
    print(k)

E
F
R
z


/var/folders/ws/5cwbl8gj2ql8jp5jlbsm15g00000gn/T/ipykernel_27149/4217589431.py:1: DeprecationWarning: NpzFile.iterkeys is deprecated in python 3, to match the removal of dict.iterkeys. Use .keys() instead.
  for k in data.iterkeys():


In [4]:
for k in data.iterkeys():
  print(f"key {k} has dimension",data[k].shape)

key E has dimension (22140, 1)
key F has dimension (22140, 15, 3)
key R has dimension (22140, 15, 3)
key z has dimension (15,)


/var/folders/ws/5cwbl8gj2ql8jp5jlbsm15g00000gn/T/ipykernel_27149/4248861679.py:1: DeprecationWarning: NpzFile.iterkeys is deprecated in python 3, to match the removal of dict.iterkeys. Use .keys() instead.
  for k in data.iterkeys():


In [5]:
R.shape

(22140, 15, 3)

In [6]:
len([r.shape for r in R])



22140

we have 22140 rows of (15,3) dimension

In [7]:
R[0].flatten().shape

(45,)

In [8]:
from scipy.spatial.distance import pdist

D = np.array([1./pdist(r) for r in R])

print(D.shape)

(22140, 105)


In [9]:
pdist(R[0]).shape



(105,)

In [10]:
from sklearn.model_selection import train_test_split

#We aim at predicting the E values given the distances
X_train, X_test, y_train, y_test = train_test_split(D, E, train_size=1000, random_state=0)




```
ls = 0.15
nl = 0.1
def k(x, z, ls=1):
    return np.exp(-(x-z)**2 / (2 * ls ** 2))

#I_n = np.broadcast_to(np.identity(K.shape[0]), (K.shape[2],K.shape[0],K.shape[1]))

i = np.identity(K.shape[0])
I_n = np.dstack([i]*K.shape[2])

K = np.array([[k(z, x, ls=ls) for z in X_train] for x in X_train])
alpha = np.linalg.pinv(K + nl*I_n) @ y_train   

#def k(x, z, ls=1):
#    return np.exp(-(x-z)**2 / (2 * ls ** 2))
```



In [27]:
## Implement KRR class
def kernelf_gaussian(x, z=None, ls=1.0):
    """Compute kernel matrix with Gaussian kernel.
    
    Args:
        x: iterable, row of kernel matrix
        z: (optional) iterable, column of kernel matrix. if None, use x
        ls: length scale
    
    """
    if z is None:
        z = x
    return np.array([[np.exp(-(this_x-this_z)**2 / (2 * ls ** 2)) for this_z in z] for this_x in x])
    
#ls = 0.15
#`nl = 0.1



   

class KRR():
    def __init__(self, ls, nl, kernelf=kernelf_gaussian):
        self.ls = ls
        self.nl = nl
        self.kernelf = kernelf


    def k(x, z, ls=1):
        return np.exp(-(x-z)**2 / (2 * ls ** 2))


    def train(self, x, y):
        self.X_train = x
        K = self.kernelf(x, ls=self.ls)
        i = np.identity(K.shape[0])
        I_n = np.dstack([i]*K.shape[2])
        self.alpha = np.linalg.pinv(K + self.nl*I_n) @ y


    def predict(self, z):
        L = self.kernelf(self.X_train, ls=self.ls, z=z)
        return L.T @ self.alpha

#krr = KRR(ls=0.15, nl=0.1)

#krr.train(X_train, y_train)


In [39]:
L = kernelf_gaussian(X_train, ls=1.0)
L.T.shape

(105, 1000, 1000)

In [41]:
alpha.shape

(1000, 105, 1)

In [42]:
L.T @ alpha

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 105 is different from 1000)

In [28]:
krr = KRR(ls=0.15, nl=0.1)

In [29]:
krr.train(X_train, y_train)

In [33]:
L

NameError: name 'L' is not defined

In [30]:
krr.predict(X_test)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 105 is different from 1000)

In [18]:
import numpy as np
ls = 0.2
nl = 0.1

def k(x, z, ls=1):
    return np.exp(-(x-z)*2 / (2 * ls * 2))


#K = [[k(z, x, ls=ls) for z in X_train] for x in X_train]
#alpha = np.linalg.inv(K) @ y_train  # @ is matrix multiplication
K = np.array([[k(z, x, ls=ls) for z in X_train] for x in X_train])
i = np.identity(K.shape[0])
I_n = np.dstack([i]*K.shape[2])
alpha = np.linalg.pinv(K + nl*I_n) @ y_train

#K + nl*I_n

In [19]:
alpha

array([[[155.88617101],
        [  7.60539231],
        [ -3.05235259],
        ...,
        [ 70.7012198 ],
        [166.63483665],
        [ 73.37694697]],

       [[145.72656378],
        [  2.53550281],
        [-19.3791244 ],
        ...,
        [ 67.58099018],
        [170.93369526],
        [ 74.34577279]],

       [[142.77339149],
        [  0.57807354],
        [-12.60402927],
        ...,
        [ 72.00279987],
        [174.91392928],
        [ 77.9943306 ]],

       ...,

       [[161.85182077],
        [  5.33231166],
        [-11.45800816],
        ...,
        [ 73.3965043 ],
        [173.96249199],
        [ 72.3504038 ]],

       [[163.73017877],
        [  3.21099639],
        [-27.41224382],
        ...,
        [ 74.39698464],
        [177.4816088 ],
        [ 76.72888395]],

       [[147.16072813],
        [  4.90951026],
        [-19.82538862],
        ...,
        [ 68.22271027],
        [171.5557015 ],
        [ 79.19323399]]])

In [25]:
X_test.shape

(21140, 105)

In [26]:
def basis_function(x, i):
    # function centered on i-th training point
    return alpha[i]*np.array([k(this_x, X_train[i], ls=ls) for this_x in x])

#def f_hat(x):
#    # total prediction
#    return np.sum([basis_function(x, i) for i in range(n)], axis=0)

In [ ]:
basis_function(X,0)

In [23]:
f_hat(X_test)

NameError: name 'n' is not defined

In [ ]:
import numpy as np
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, make_scorer

class KRR():

  def __init__(self, alpha=np.logspace(-12, -6, 7), gamma=np.logspace(-4, -1, 1000)):

    self.scorer = make_scorer(mean_squared_error, greater_is_better=False)
    self.best_params_ = None
    self.best_score_ = None

  
    self.clf = GridSearchCV(KernelRidge(kernel='rbf', gamma=1.0), cv=KFold(n_splits=5, shuffle=True),
                            param_grid={"alpha": alpha, "gamma": gamma},
                            verbose=1, n_jobs=-1, scoring=self.scorer)

  def train(self, X, Y):

    self.clf.fit(X, Y)
    self.best_params_ = self.clf.best_params_
    self.best_score_ = self.clf.best_score_


  def predict(self, X):

    return self.clf.predict(X)


def gridSearch():
  max_accuracy = 0
  param_grid = {"alpha": alpha, "gamma": gamma}
  




In [ ]:
from krr import KRR

krr = KRR()
krr.train(X_train, y_train)

print('Best: alpha = {}, gamma = {}'.format(krr.best_params_['alpha'], krr.best_params_['gamma']))

In [ ]:
y_train_predicted = krr.predict(X_train)
y_test_predicted = krr.predict(X_test)

In [ ]:
mae = lambda X, Y: np.mean(np.absolute((X - Y)))
rmse = lambda X, Y: np.sqrt(np.mean(np.square((X - Y))))

In [ ]:
train_mae = mae(y_train_predicted, y_train)
train_rmse = rmse(y_train_predicted, y_train)
test_mae = mae(y_test_predicted, y_test)
test_rmse = rmse(y_test_predicted, y_test)

print('Train: MAE = {:.3f} kcal/mol, RMSE = {:.3f} kcal/mol'. format(train_mae, train_rmse))
print('Tset: MAE = {:.3f} kcal/mol, RMSE = {:.3f} kcal/mol'. format(test_mae, test_rmse))

In [ ]:
from matplotlib import pyplot as plt
 
plt.scatter(y_train, y_train_predicted, marker='.', color='green', zorder=2)
plt.scatter(y_test, y_test_predicted, marker='.', color='blue', zorder=1)
plt.title('MAE = {:.3f} kcal/mol, RMSE = {:.3f} kcal/mol'.format(test_mae, test_rmse))
plt.savefig('krr.png', dpi=600)
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model

alpha=np.logspace(-12, 0, 13)
clf = GridSearchCV(linear_model.Ridge(alpha=1), cv=5, param_grid={"alpha": alpha}, 
                   verbose=1, n_jobs=-1, scoring="neg_mean_squared_error")

clf.fit(X_train, y_train)

print('Best alpha: {:}'.format(clf.best_params_['alpha']))

y_train_predicted_clf = clf.predict(X_train)
y_test_predicted_clf = clf.predict(X_test)

train_mae_clf = mae(y_train_predicted_clf, y_train)
train_rmse_clf = rmse(y_train_predicted_clf, y_train)
test_mae_clf = mae(y_test_predicted_clf, y_test)
test_rmse_clf = rmse(y_test_predicted_clf, y_test)

print('Train: MAE = {:.3f} kcal/mol, RMSE = {:.3f} kcal/mol'. format(train_mae_clf, train_rmse_clf))
print('Tset: MAE = {:.3f} kcal/mol, RMSE = {:.3f} kcal/mol'. format(test_mae_clf, test_rmse_clf))

plt.scatter(y_train, y_train_predicted_clf, marker='.', color='green', zorder=2)
plt.scatter(y_test, y_test_predicted_clf, marker='.', color='blue', zorder=1)
plt.title('MAE = {:.3f} kcal/mol, RMSE = {:.3f} kcal/mol'.format(test_mae_clf, test_rmse_clf))
plt.savefig('rr.png', dpi=600)
plt.show()